In [1]:
import statsmodels.api as sm
import pandas as pd
from sklearn.metrics import roc_auc_score

from src.radiomics.data import load_data, get_formatted_data

In [2]:
df = pd.read_csv("/home/valentin/python_wkspce/plc_segmentation/data/clinical_info_updated.csv")

In [3]:
list(df.columns)

['patient_id',
 'is_chuv',
 'plc_status',
 'pT',
 'pN',
 'M',
 'Stage',
 'pathologic_type',
 'new_cases',
 'LymphangitisCT',
 'sexe',
 'age']

In [4]:
df = load_data(
    "/home/valentin/python_wkspce/plc_segmentation/data/processed/radiomics/extracted_features.csv",
    "/home/valentin/python_wkspce/plc_segmentation/data/clinical_info_updated.csv",
    clinical_info=["plc_status", "LymphangitisCT"],
)

In [5]:
features, outcomes = get_formatted_data(df,
                                        modality="PT",
                                        voi="GTV_L",
                                        return_df=True,
                                        clinical_info=["LymphangitisCT"])


In [6]:
plc_status_ct = features[["LymphangitisCT"]]

In [7]:
# patient_to_keep = [p for p in outcomes.index if "Patient" in p]

In [8]:
# features = features[features.index.isin(patient_to_keep)]
# outcomes = outcomes[outcomes.index.isin(patient_to_keep)]

In [9]:
suvmax = features[["original_firstorder_Maximum"]]

In [10]:
log_reg = sm.Logit(outcomes, suvmax).fit()

Optimization terminated successfully.
         Current function value: 0.659412
         Iterations 5


In [11]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:             plc_status   No. Observations:                  106
Model:                          Logit   Df Residuals:                      105
Method:                           MLE   Df Model:                            0
Date:                Tue, 15 Feb 2022   Pseudo R-squ.:                 0.02344
Time:                        09:59:58   Log-Likelihood:                -69.898
converged:                       True   LL-Null:                       -71.575
Covariance Type:            nonrobust   LLR p-value:                       nan
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
original_firstorder_Maximum     0.1763      0.069      2.549      0.011       0.041       0.312


In [12]:
roc_auc_score(outcomes, suvmax)

0.6330749354005168

In [13]:
plc_status_ct

,LymphangitisCT
patient_id,
Case_1,0.0
Case_10,0.0
Case_11,0.0
Case_12,0.0
Case_13,0.0
...,...
PatientLC_90,2.0
PatientLC_91,2.0
PatientLC_92,1.0


In [23]:
roc_auc_score(outcomes[~plc_status_ct["LymphangitisCT"].isna()], plc_status_ct[~plc_status_ct["LymphangitisCT"].isna()])

0.6778058007566203